# Multi-Head Attention
:label:`sec_multihead-attention`


In practice,
given the same set of queries, keys, and values
we may want our model to
combine knowledge from
different behaviors of the same attention mechanism,
such as capturing dependencies of various ranges (e.g., shorter-range vs. longer-range)
within a sequence.
Thus, 
it may be beneficial 
to allow our attention mechanism
to jointly use different representation subspaces
of queries, keys, and values.



To this end,
instead of performing a single attention pooling,
queries, keys, and values
can be transformed
with $h$ independently learned linear projections.
Then these $h$ projected queries, keys, and values
are fed into attention pooling in parallel.
In the end,
$h$ attention pooling outputs
are concatenated and 
transformed with another learned linear projection
to produce the final output.
This design
is called *multi-head attention*,
where each of the $h$ attention pooling outputs
is a *head* :cite:`Vaswani.Shazeer.Parmar.ea.2017`.
Using fully-connected layers
to perform learnable linear transformations,
:numref:`fig_multi-head-attention`
describes multi-head attention.

![Multi-head attention, where multiple heads are concatenated then linearly transformed.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/multi-head-attention.svg)
:label:`fig_multi-head-attention`




## Model

Before providing the implementation of multi-head attention,
let us formalize this model mathematically.
Given a query $\mathbf{q} \in \mathbb{R}^{d_q}$,
a key $\mathbf{k} \in \mathbb{R}^{d_k}$,
and a value $\mathbf{v} \in \mathbb{R}^{d_v}$,
each attention head $\mathbf{h}_i$  ($i = 1, \ldots, h$)
is computed as

$$\mathbf{h}_i = f(\mathbf W_i^{(q)}\mathbf q, \mathbf W_i^{(k)}\mathbf k,\mathbf W_i^{(v)}\mathbf v) \in \mathbb R^{p_v},$$

where learnable parameters
$\mathbf W_i^{(q)}\in\mathbb R^{p_q\times d_q}$,
$\mathbf W_i^{(k)}\in\mathbb R^{p_k\times d_k}$
and $\mathbf W_i^{(v)}\in\mathbb R^{p_v\times d_v}$,
and
$f$ is attention pooling,
such as
additive attention and scaled dot-product attention
in :numref:`sec_attention-scoring-functions`.
The multi-head attention output
is another linear transformation via 
learnable parameters
$\mathbf W_o\in\mathbb R^{p_o\times h p_v}$
of the concatenation of $h$ heads:

$$\mathbf W_o \begin{bmatrix}\mathbf h_1\\\vdots\\\mathbf h_h\end{bmatrix} \in \mathbb{R}^{p_o}.$$

Based on this design,
each head may attend to different parts of the input.
More sophisticated functions than the simple weighted average
can be expressed.


In [ ]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.11.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26

%maven ai.djl.mxnet:mxnet-engine:0.11.0
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-backport

In [ ]:
%load ../utils/plot-utils
%load ../utils/Functions.java
%load ../utils/PlotUtils.java
%load ../utils/AttentionUtils.java

In [ ]:
import ai.djl.ndarray.*;
import ai.djl.ndarray.types.DataType;
import ai.djl.ndarray.types.Shape;
import ai.djl.nn.AbstractBlock;
import ai.djl.nn.Parameter;
import ai.djl.nn.core.Linear;
import ai.djl.nn.norm.Dropout;
import ai.djl.training.ParameterStore;
import ai.djl.util.PairList;

In [ ]:
NDManager manager = NDManager.newBaseManager(Functions.tryGpu(0));

To allow for parallel computation of multiple heads,
the below `MultiHeadAttention` class uses two transposition functions as defined below.
Specifically,
the `transposeOutput` function reverses the operation
of the `transposeQkv` function.

In [ ]:
public static NDArray transposeQkv(NDArray X, int numHeads) {
    // Shape of input `X`:
    // (`batchSize`, no. of queries or key-value pairs, `numHiddens`).
    // Shape of output `X`:
    // (`batchSize`, no. of queries or key-value pairs, `numHeads`,
    // `numHiddens` / `numHeads`)
    X = X.reshape(X.getShape().get(0), X.getShape().get(1), numHeads, -1);

    // Shape of output `X`:
    // (`batchSize`, `numHeads`, no. of queries or key-value pairs,
    // `numHiddens` / `numHeads`)
    X = X.transpose(0, 2, 1, 3);

    // Shape of `output`:
    // (`batchSize` * `numHeads`, no. of queries or key-value pairs,
    // `numHiddens` / `numHeads`)
    return X.reshape(-1, X.getShape().get(2), X.getShape().get(3));
}

public static NDArray transposeOutput(NDArray X, int numHeads) {
    X = X.reshape(-1, numHeads, X.getShape().get(1), X.getShape().get(2));
    X = X.transpose(0, 2, 1, 3);
    return X.reshape(X.getShape().get(0), X.getShape().get(1), -1);
}

## Implementation

In our implementation,
we choose the scaled dot-product attention
for each head of the multi-head attention.
To avoid significant growth
of computational cost and parameterization cost,
we set
$p_q = p_k = p_v = p_o / h$.
Note that $h$ heads
can be computed in parallel
if we set
the number of outputs of linear transformations
for the query, key, and value
to $p_q h = p_k h = p_v h = p_o$.
In the following implementation,
$p_o$ is specified via the argument `numHiddens`.


In [ ]:
public class MultiHeadAttention extends AbstractBlock {
    private static final byte VERSION = 1;
    private int numHeads;
    public DotProductAttention attention;
    private Linear W_k;
    private Linear W_q;
    private Linear W_v;
    private Linear W_o;
    private Dropout dropout;

    public MultiHeadAttention(int numHiddens, int numHeads, float dropout, boolean useBias) {
        super(VERSION);
        this.numHeads = numHeads;

        attention = new DotProductAttention(dropout);

        this.W_q = Linear.builder().setUnits(numHiddens).optBias(useBias).build();
        this.addChildBlock("W_q", this.W_q);

        this.W_k = Linear.builder().setUnits(numHiddens).optBias(useBias).build();
        this.addChildBlock("W_k", this.W_k);

        this.W_v = Linear.builder().setUnits(numHiddens).optBias(useBias).build();
        this.addChildBlock("W_v", this.W_v);

        this.W_o = Linear.builder().setUnits(numHiddens).optBias(useBias).build();
        this.addChildBlock("W_o", this.W_o);

        this.dropout = Dropout.builder().optRate(dropout).build();
        this.addChildBlock("dropout", this.dropout);
    }

    @Override
    protected NDList forwardInternal(
            ParameterStore parameterStore,
            NDList inputs,
            boolean training,
            PairList<String, Object> params) {
        // Shape of `queries`, `keys`, or `values`:
        // (`batchSize`, no. of queries or key-value pairs, `numHiddens`)
        // Shape of `validLens`:
        // (`batchSize`,) or (`batchSize`, no. of queries)
        // After transposing, shape of output `queries`, `keys`, or `values`:
        // (`batchSize` * `numHeads`, no. of queries or key-value pairs,
        // `numHiddens` / `numHeads`)
        NDArray queries = inputs.get(0);
        NDArray keys = inputs.get(1);
        NDArray values = inputs.get(2);
        NDArray validLens = inputs.get(3);

        queries =
                transposeQkv(
                        W_q.forward(parameterStore, new NDList(queries), training, params).get(0),
                        this.numHeads);
        keys =
                transposeQkv(
                        W_k.forward(parameterStore, new NDList(keys), training, params).get(0),
                        this.numHeads);
        values =
                transposeQkv(
                        W_v.forward(parameterStore, new NDList(values), training, params).get(0),
                        this.numHeads);

        if (validLens != null) {
            // On axis 0, copy the first item (scalar or vector) for
            // `numHeads` times, then copy the next item, and so on
            validLens = validLens.repeat(0, this.numHeads);
        }

        // Shape of `output`: (`batchSize` * `numHeads`, no. of queries,
        // `numHiddens` / `numHeads`)
        NDArray output =
                this.attention
                        .forward(
                                parameterStore,
                                new NDList(queries, keys, values, validLens),
                                training,
                                params)
                        .get(0);

        // Shape of `outputConcat`:
        // (`batchSize`, no. of queries, `numHiddens`)
        NDArray outputConcat = transposeOutput(output, this.numHeads);
        return new NDList(
                this.W_o.forward(parameterStore, new NDList(outputConcat), training, params)
                        .get(0));
    }

    @Override
    public Shape[] getOutputShapes(Shape[] inputShapes) {
        throw new UnsupportedOperationException("Not implemented");
    }

    @Override
    public void initializeChildBlocks(NDManager manager, DataType dataType, Shape... inputShapes) {}
}

Let us test our implemented `MultiHeadAttention` class
using a toy example where keys and values are the same.
As a result,
the shape of the multi-head attention output
is (`batchSize`, `numQueries`, `numHiddens`).


In [ ]:
int numHiddens = 100;
int numHeads = 5;
MultiHeadAttention attention = new MultiHeadAttention(numHiddens, numHeads, 0.5f, false);

In [ ]:
int batchSize = 2;
int numQueries = 4;
int numKvpairs = 6;
NDArray validLens = manager.create(new float[] {3, 2});
NDArray X = manager.ones(new Shape(batchSize, numQueries, numHiddens));
NDArray Y = manager.ones(new Shape(batchSize, numKvpairs, numHiddens));

System.out.println(
        attention
                .forward(
                        new ParameterStore(manager, false),
                        new NDList(X, Y, Y, validLens),
                        false)
                .get(0)
                .getShape());

## Summary

* Multi-head attention combines knowledge of the same attention pooling via different representation subspaces of queries, keys, and values.
* To compute multiple heads of multi-head attention in parallel, proper tensor manipulation is needed.



## Exercises

1. Visualize attention weights of multiple heads in this experiment.
1. Suppose that we have a trained model based on multi-head attention and we want to prune least important attention heads to increase the prediction speed. How can we design experiments to measure the importance of an attention head?
